In [43]:
from robot import RobotDict
from decision_vars import *
import casadi as ca

H = 5     # num MPC steps
R = 0.1   # control weight in cost function
dt = 0.01 # time step in sec for discretized dynamics

solver_opts = {"print_time":False,
               "ipopt.print_level":0,
               "ipopt.tol":1.0e-5,
               "ipopt.acceptable_constr_viol_tol":2.0e-04,
               "ipopt.linear_solver":'MUMPS',
               "ipopt.warm_start_init_point":"yes",
               "ipopt.warm_start_bound_frac":1.0e-09,
               "ipopt.warm_start_bound_push":1.0e-09,
              }

robots = RobotDict("config_files/franka.yaml", ["config_files/contact.yaml", "config_files/free_space.yaml"], {})
robot = robots.param_dict['contact']

robot.build_disc_dyn(dt, {}) # Rebuild dyn with larger step size

nq = robot.nq
nx = 2*robot.nq
qd = np.ones((nq,1))

# Make the parameters, which are fixed numerical values which can be updated in each solve
params_init = dict(xi_t = np.zeros((2*robot.nq, 1)))
params = param_set(params_init, symb_type = ca.SX.sym)

# Make the decision variables, which are optimized
vars_init =  dict(tau = np.zeros((nq, H)),
                  xi = np.zeros((nx, H-1)))
vars_lb = dict(tau = -np.ones((nq, H)))
vars_ub = {k:-v for k,v in vars_lb.items()}
vars = decision_var_set(x0 = vars_init, lb = vars_lb, ub = vars_ub)

g = [] # lists for the constraints and it's bounds
J = 0  # objective function

dyn = robot.disc_dyn
xi = ca.horzcat(np.zeros((nx,1)), vars['xi']) # some issue with parameters, just setting initial state to zero
print(vars["tau"].shape)
xi_next = dyn(xi, vars['tau'])
print(xi_next.shape)
g += [ca.reshape(xi_next[:,:-1]-vars['xi'][:,:], nx*(H-1), 1)]
J += ca.sumsqr(qd-xi_next[:nq,:])
J += ca.sumsqr(xi_next[nq:,:])
J += ca.sumsqr(R*vars['tau'])

x, x_lb, x_ub = vars.get_dec_vectors()
x0 = vars.get_x0()

args = dict(x0=x0, lbx=x_lb, ubx=x_ub)
prob = dict(f=J, x=x, g=ca.vertcat(*g))#, p=params.get_vector())
solver = ca.nlpsol('solver', 'ipopt', prob, solver_opts)
num_particles = 30
num_elites = 1
mu = np.zeros(H*nq)
cov = np.eye(H*nq)
samples = np.random.multivariate_normal(mu, cov, size=num_particles)
formatted_samples = np.zeros((num_particles, nq, H))
for i in range(num_particles):
    for t in range(H):
        u = samples[i, slice(t*nq, (t+1)*nq)]
        formatted_samples[i][:, t] = u
    #print(formatted_samples[i])
#print(formatted_samples.shape)
x_0 = np.zeros((nx, H))
rollouts = np.zeros((num_particles, nx, H)) # np tensor containing state rollouts for a planning horizon
for i in range(num_particles):
    rollouts[i] = dyn(x_0, formatted_samples[i])

cost = np.zeros((num_particles, 1))
for i in range(num_particles):
    cost[i,:] = ca.sumsqr(qd-rollouts[i][:nq,:]) + ca.sumsqr(formatted_samples[i])

minimum_cost_index = np.argmin(cost, axis=0)
#print(cost)
#print(minimum_cost_index)
best_state_traj = rollouts[minimum_cost_index].reshape(nx, H)
#vars['xi'].set_x0(best_state_traj) # set initial value of dec var to best trajectory
#vars['tau'].set_x0(best_input_traj)
#args['x0'] = vars.get_x0() # warm start from prev solution

best_input_traj = formatted_samples[minimum_cost_index].reshape(nq, H)
#print(best_state_traj.shape)
#print(best_input_traj.shape)
warm_start = ca.vertcat(best_state_traj, best_input_traj)
warm_start = ca.reshape(warm_start, 3*nq*H, 1)[:91]
#print(warm_start.shape)

    
#print(rollouts[0][:nq,:].shape)
#J = ca.sumsqr(qd-rollouts[0][:nq, :]) + ca.sumsqr(formatted_samples[0])
#print(J)
#print(rollouts.shape)        
#print(dyn(x_0, vars["tau"]).shape)


loading model: contact
Building robot model with:
  contact model(s):  ['contact_1']
  optimization pars: {}
  estimation pars:   {}
<class 'pinocchio.pinocchio_pywrap_casadi.SE3'>
(DM([0, 0, 0]), DM(
[[1, 0, 0], 
 [0, 1, 0], 
 [0, 0, 1]]))
loading model: free-space
Building robot model with:
  contact model(s):  ['contact_1']
  optimization pars: {}
  estimation pars:   {}
<class 'pinocchio.pinocchio_pywrap_casadi.SE3'>
(DM([0, 0, 0]), DM(
[[1, 0, 0], 
 [0, 1, 0], 
 [0, 0, 1]]))
(7, 5)
(14, 5)


In [84]:
#def rollout_traj(dyn, u_samples):
rollout = np.zeros((num_particles, nx, H))
#print(rollout[0][:,0].shape)

#print(x0.shape)
for i in range(num_particles):
    x0 = np.zeros((nx,1))
    
    for t in range(H):
        #print(formatted_samples[i][:,t].reshape(nq,1).shape)
        
        #print(np.array(dyn(x0, formatted_samples[i][:,t])).reshape(nx))
        rollout[i][:,t] = np.array(dyn(x0, formatted_samples[i][:,t])).reshape(nx)
        x0 = rollout[i][:,t]
    
        
        #print(rollout[i][:,t])
        
print(rollout.shape)
        
trial = dyn(np.zeros((nx,1)), formatted_samples[0][:,0])
trial_next = dyn(trial, formatted_samples[0][:,1])
#print(trial)
#print(trial_next)


(30, 14, 5)
